In [6]:
import sys
sys.path.append('/opt/notebooks/polytrader')

In [7]:
from agents.polymarket.polymarket import Polymarket

In [8]:
p = Polymarket()

In [9]:
game = p.get_market_by_slug('nfl-phi-bal-2024-12-01')
outcome = game.outcomes[0]

In [ ]:
from agents.trackers.market_tracker import MarketChannel, MarketMessage
import asyncio
async def track(market_obj):
    # Initialize market channel
    market_channel = MarketChannel(market_obj)

    # Example callback function
    def handle_market_update(outcome: str, message: MarketMessage):
        print(f'Market update for {outcome}: {message}')
        
    await market_channel.connect()

    # Add callback for specific assets
    market_channel.add_outcome_callback(outcome, handle_market_update)

    try:
        # Keep connection alive
        while True:
            await asyncio.sleep(1)
    except KeyboardInterrupt:
        await market_channel.close()

In [ ]:
await track(game)